Introduction a SQLAlchemy 
=======================

## ¿Qué es SQLAlchemy?

* SQLAlchemy es una librería de Python que provee de un interface para bases de datos relacionales como:
    - Oracle
    - MySQL
    - PostgreSQL
    - DB2
    - SQLite
* SQLAlchemy incluye además un object-relational mapper (ORM).

## ¿Por qué usar SQLAlchemy?

* Portabilidad y abstracción: el mismo código se puede emplear para distinstas bases de datos
* Seguridad: no es necesario SQL injections
* Orientado a objetos: el trabajo se realiza con objetos y no con tablas 
* Rendimiento: las consultas pueden reutilizarse
* Flexibilidad: puede realizarse casi cualquier cosa

## Instalación

In [1]:
!pip install sqlalchemy

In [2]:
!conda install sqlalchemy

"conda" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


## Conexión con bases de datos

Para poder interactuar con una base de datos es necesario conocer las credenciales para poder acceder.

La sintaxis de la ruta de conexión a una base de datos utiliza la siguiente sintaxis:

```<dialecto><controlador>://<usuario>:<contraseña>@<ruta del servidor>:<puerto>/<base de datos>```

Para mayor información, consultar en el [link](http://docs.sqlalchemy.org/en/latest/core/engines.html#database-urls).

## Tipos de datos

La lista completa está disponible en el [link](https://docs.sqlalchemy.org/en/13/core/type_basics.html).

| SQLAlchemy | Python | SQL |
| ---------- | ---------- |  ---------- |
| BigInteger | int | BIGINT|
| Boolean | bool | BOOLEAN or SMALLINT |
|Date | datetime.date | Date (SQLite: String)|
| DateTime | datetime.datetime | DATETIME (SQLite: String)|
| Enum | str | ENUM or VARCHAR| 
| Float | float or Decimal | FLOAT or REAL|
|Integer |int | Integer|
|Interval | datetime.timedelta | INTERVAL or DATE from epoch| 
| LargeBinary | byte | BLOB or BYTEA |
| Numeric | decimal.Decimal | NUMERIC or DECIMAL |
| Unicode | unicode | UNICODE or VARCHAR |
| Text | str | CLOB or TEXT | 
| Time | datetime.time | DATETIME | 

### Inicio

In [3]:
import sqlalchemy

# Connexion a la base de datos
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:',echo=True)

In [4]:
!ls -la

"ls" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


### Creación de Tablas

Para la creación de tablas se usa un objeto base: 

In [5]:
from sqlalchemy import MetaData

# Metadata si se utilizan tablas
metadata = MetaData()

En SQLAlchemy las tablas se crean medinate las clases Columms:
* Columns ofrece distintos tipos de datos (Integer, Float, etc.)
* Columns ofrece relaciones entre tablas (Primary Key, Foreign Key, etc.)

In [6]:
from sqlalchemy import Column, Integer, String, Table

# Definicion como tablas     
users = Table('users', metadata,
              Column('id', Integer, primary_key=True),
              Column('name', String),
              Column('fullname', String),
              Column('password', String))                     

SQLAlchemy puede crear las tablas 

In [7]:
metadata.create_all(engine)

2021-07-13 20:57:22,387 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-13 20:57:22,389 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2021-07-13 20:57:22,390 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-13 20:57:22,392 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2021-07-13 20:57:22,393 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-13 20:57:22,396 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2021-07-13 20:57:22,397 INFO sqlalchemy.engine.Engine [no key 0.00127s] ()
2021-07-13 20:57:22,399 INFO sqlalchemy.engine.Engine COMMIT


##### SQL:

```SQL
CREATE TABLE users (
    id INTEGER NOT NULL, name VARCHAR,
    fullname VARCHAR,
    password VARCHAR,
    PRIMARY KEY (id)
```

### Relationships 

Multiples tablas pueden ser realionadas usando la función `ForeignKey`.

In [8]:
from sqlalchemy import ForeignKey
    
addresses = Table('addresses', metadata,
                  Column('id', Integer, primary_key=True),
                  Column('user_id', None, ForeignKey('users.id')),
                  Column('email_address', String, nullable=False))

metadata.create_all(engine)

2021-07-13 20:57:22,429 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-13 20:57:22,431 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2021-07-13 20:57:22,432 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-13 20:57:22,433 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("addresses")
2021-07-13 20:57:22,434 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-13 20:57:22,437 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("addresses")
2021-07-13 20:57:22,439 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-13 20:57:22,441 INFO sqlalchemy.engine.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	user_id INTEGER, 
	email_address VARCHAR NOT NULL, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2021-07-13 20:57:22,442 INFO sqlalchemy.engine.Engine [no key 0.00106s] ()
2021-07-13 20:57:22,445 INFO sqlalchemy.engine.Engine COMMIT


### Inicio de Session

In [9]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

#Inicio de la sesion
session = Session()

### Inserción de datos

In [10]:
paco_user = users.insert().values(name='Paco', fullname='Francisco', password='25051970')

Datos como diccionario

In [11]:
paco_user.compile().params

{'name': 'Paco', 'fullname': 'Francisco', 'password': '25051970'}

Consolidación de datos

In [12]:
# Se instancia una conexión
conn = engine.connect()

result = conn.execute(paco_user)

2021-07-13 20:57:22,703 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2021-07-13 20:57:22,704 INFO sqlalchemy.engine.Engine [generated in 0.00204s] ('Paco', 'Francisco', '25051970')
2021-07-13 20:57:22,707 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
result.inserted_primary_key

(1,)

### Multiples inserciones

In [14]:
result = conn.execute(users.insert(), [
    {'name': 'Carla', 'fullname': 'Carla', 'password': '$5&02qq'},
    {'name': 'Manu', 'fullname': 'Manuela', 'password': 'lqoeoeee'},
    {'name': 'Lu', 'fullname': 'Lucia', 'password': 'fjakclff'}
])

result1 = conn.execute(addresses.insert(), [
    {'user_id': 2, 'email_address': 'carla1?22@gmail.com'},
    {'user_id': 4, 'email_address': 'luci.mendo@gmail.com'},
    {'user_id': 1, 'email_address': 'paco84@hotmail.com'}
])

2021-07-13 20:57:22,760 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2021-07-13 20:57:22,762 INFO sqlalchemy.engine.Engine [generated in 0.00266s] (('Carla', 'Carla', '$5&02qq'), ('Manu', 'Manuela', 'lqoeoeee'), ('Lu', 'Lucia', 'fjakclff'))
2021-07-13 20:57:22,765 INFO sqlalchemy.engine.Engine COMMIT
2021-07-13 20:57:22,769 INFO sqlalchemy.engine.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2021-07-13 20:57:22,770 INFO sqlalchemy.engine.Engine [generated in 0.00190s] ((2, 'carla1?22@gmail.com'), (4, 'luci.mendo@gmail.com'), (1, 'paco84@hotmail.com'))
2021-07-13 20:57:22,772 INFO sqlalchemy.engine.Engine COMMIT


### Update

In [15]:
up = users.update().where(users.c.name == 'Lu').\
        values(password='iqq@3399')

conn.execute(up)

2021-07-13 20:57:22,798 INFO sqlalchemy.engine.Engine UPDATE users SET password=? WHERE users.name = ?
2021-07-13 20:57:22,806 INFO sqlalchemy.engine.Engine [generated in 0.00204s] ('iqq@3399', 'Lu')
2021-07-13 20:57:22,808 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
from sqlalchemy import bindparam

up = users.update().\
             where(users.c.fullname == bindparam('oldfullname')).\
             values(fullname=bindparam('newfullname'))

conn.execute(up, [
     {'oldfullname':'Francisco', 'newfullname':'Francisco Javier'},
     ])

2021-07-13 20:57:22,842 INFO sqlalchemy.engine.Engine UPDATE users SET fullname=? WHERE users.fullname = ?
2021-07-13 20:57:22,844 INFO sqlalchemy.engine.Engine [generated in 0.00189s] ('Francisco Javier', 'Francisco')
2021-07-13 20:57:22,846 INFO sqlalchemy.engine.Engine COMMIT


Actualización con diccionarios

In [17]:
up =  addresses.update().\
        values({
            addresses.c.email_address:'carlota12@gmail.com'
        }).\
        where(addresses.c.email_address.startswith('ca%'))

conn.execute(up)

2021-07-13 20:57:22,885 INFO sqlalchemy.engine.Engine UPDATE addresses SET email_address=? WHERE (addresses.email_address LIKE ? || '%')
2021-07-13 20:57:22,887 INFO sqlalchemy.engine.Engine [generated in 0.00219s] ('carlota12@gmail.com', 'ca%')
2021-07-13 20:57:22,889 INFO sqlalchemy.engine.Engine COMMIT


### Select

In [18]:
from sqlalchemy.sql import select

# Obtenemos los usuarios
s = select([users])
result = conn.execute(s)

2021-07-13 20:57:22,919 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname, users.password 
FROM users
2021-07-13 20:57:22,920 INFO sqlalchemy.engine.Engine [generated in 0.00173s] ()


In [19]:
for row in result:
    print(row)

(1, 'Paco', 'Francisco Javier', '25051970')
(2, 'Carla', 'Carla', '$5&02qq')
(3, 'Manu', 'Manuela', 'lqoeoeee')
(4, 'Lu', 'Lucia', 'iqq@3399')


SQLAlchemy tiene también métodos fetchone() y fetchall() 

In [20]:
result = conn.execute(s)
row = result.fetchone()

2021-07-13 20:57:22,973 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname, users.password 
FROM users
2021-07-13 20:57:22,975 INFO sqlalchemy.engine.Engine [cached since 0.0572s ago] ()


In [21]:
for elem in row:
    print(elem)

1
Paco
Francisco Javier
25051970


In [22]:
rows = result.fetchall()

In [23]:
for elem in rows:
    print(elem)

(2, 'Carla', 'Carla', '$5&02qq')
(3, 'Manu', 'Manuela', 'lqoeoeee')
(4, 'Lu', 'Lucia', 'iqq@3399')


Con la el atributo `c` se puede acceder a una columna en concreto

In [24]:
s = select([users.c.name, users.c.fullname])
result = conn.execute(s)

for row in result:
    print(row)

2021-07-13 20:57:23,085 INFO sqlalchemy.engine.Engine SELECT users.name, users.fullname 
FROM users
2021-07-13 20:57:23,086 INFO sqlalchemy.engine.Engine [generated in 0.00129s] ()
('Paco', 'Francisco Javier')
('Carla', 'Carla')
('Manu', 'Manuela')
('Lu', 'Lucia')


Selección de varias tablas

In [25]:
for row in conn.execute(select([users, addresses])):
    print(row)

2021-07-13 20:57:23,124 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname, users.password, addresses.id AS id_1, addresses.user_id, addresses.email_address 
FROM users, addresses
2021-07-13 20:57:23,125 INFO sqlalchemy.engine.Engine [generated in 0.00144s] ()
(1, 'Paco', 'Francisco Javier', '25051970', 1, 2, 'carlota12@gmail.com')
(1, 'Paco', 'Francisco Javier', '25051970', 2, 4, 'luci.mendo@gmail.com')
(1, 'Paco', 'Francisco Javier', '25051970', 3, 1, 'paco84@hotmail.com')
(2, 'Carla', 'Carla', '$5&02qq', 1, 2, 'carlota12@gmail.com')
(2, 'Carla', 'Carla', '$5&02qq', 2, 4, 'luci.mendo@gmail.com')
(2, 'Carla', 'Carla', '$5&02qq', 3, 1, 'paco84@hotmail.com')
(3, 'Manu', 'Manuela', 'lqoeoeee', 1, 2, 'carlota12@gmail.com')
(3, 'Manu', 'Manuela', 'lqoeoeee', 2, 4, 'luci.mendo@gmail.com')
(3, 'Manu', 'Manuela', 'lqoeoeee', 3, 1, 'paco84@hotmail.com')
(4, 'Lu', 'Lucia', 'iqq@3399', 1, 2, 'carlota12@gmail.com')
(4, 'Lu', 'Lucia', 'iqq@3399', 2, 4, 'luci.mendo@gmail.com'

##### SQL:

```SQL
SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address
FROM users, addresses
```

In [26]:
# Para mejorar la sentencia anterior
s = select([users, addresses]).where(users.c.id == addresses.c.user_id)
for row in conn.execute(s):
    print(row)

2021-07-13 20:57:23,163 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname, users.password, addresses.id AS id_1, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2021-07-13 20:57:23,165 INFO sqlalchemy.engine.Engine [generated in 0.00157s] ()
(2, 'Carla', 'Carla', '$5&02qq', 1, 2, 'carlota12@gmail.com')
(4, 'Lu', 'Lucia', 'iqq@3399', 2, 4, 'luci.mendo@gmail.com')
(1, 'Paco', 'Francisco Javier', '25051970', 3, 1, 'paco84@hotmail.com')


##### SQL:

```SQL
SELECT users.id, users.name, users.fullname, addresses.id,
   addresses.user_id, addresses.email_address
FROM users, addresses
WHERE users.id = addresses.user_id
```

Uso de sentencias SQL

In [27]:
from sqlalchemy.sql import text

s = text("SELECT users.id, addresses.id, users.id, "
     "users.name, addresses.email_address AS email "
     "FROM users JOIN addresses ON users.id=addresses.user_id "
     "WHERE users.id = 1").columns(
        users.c.id,
        addresses.c.id,
        addresses.c.user_id,
        users.c.name,
        addresses.c.email_address
     )

result = conn.execute(s)

2021-07-13 20:57:23,192 INFO sqlalchemy.engine.Engine SELECT users.id, addresses.id, users.id, users.name, addresses.email_address AS email FROM users JOIN addresses ON users.id=addresses.user_id WHERE users.id = 1
2021-07-13 20:57:23,194 INFO sqlalchemy.engine.Engine [generated in 0.00169s] ()


In [28]:
result.fetchall()

[(1, 3, 1, 'Paco', 'paco84@hotmail.com')]

Una vez realizadas las consultas cerramos el objeto

In [29]:
result.close()

### Conjunctions

In [30]:
from sqlalchemy.sql import and_, or_, not_

sent = and_(
    users.c.name.like('c%'),
    users.c.id == addresses.c.user_id,
    or_(
        addresses.c.email_address == 'carla1?22@gmail.com',
        addresses.c.email_address == 'luci.mendo@gmail.com' 
    ),
    not_(users.c.id > 3)
    )
print(sent)

users.name LIKE :name_1 AND users.id = addresses.user_id AND (addresses.email_address = :email_address_1 OR addresses.email_address = :email_address_2) AND users.id <= :id_1


In [31]:
s = select([(users.c.fullname +
               ", " + addresses.c.email_address)]).\
        where(
           and_(
               users.c.id == addresses.c.user_id,
               or_(
                  addresses.c.email_address.like('%@gmail.com'),
                  addresses.c.email_address.like('%@hotmail.com')
               )
           )
        )
conn.execute(s).fetchall()

2021-07-13 20:57:23,294 INFO sqlalchemy.engine.Engine SELECT users.fullname || ? || addresses.email_address AS anon_1 
FROM users, addresses 
WHERE users.id = addresses.user_id AND (addresses.email_address LIKE ? OR addresses.email_address LIKE ?)
2021-07-13 20:57:23,296 INFO sqlalchemy.engine.Engine [generated in 0.00193s] (', ', '%@gmail.com', '%@hotmail.com')


[('Carla, carlota12@gmail.com',),
 ('Lucia, luci.mendo@gmail.com',),
 ('Francisco Javier, paco84@hotmail.com',)]

### Funciones

In [32]:
from sqlalchemy.sql import func

In [33]:
conn.execute(
    select([
            func.max(addresses.c.email_address, type_=String)
           ])
     ).scalar() # Primer elemento del primer resultado

2021-07-13 20:57:23,356 INFO sqlalchemy.engine.Engine SELECT max(addresses.email_address) AS max_1 
FROM addresses
2021-07-13 20:57:23,358 INFO sqlalchemy.engine.Engine [generated in 0.00201s] ()


'paco84@hotmail.com'

In [34]:
conn.execute(
    select([
            func.min(addresses.c.user_id)
           ])
     ).scalar()

2021-07-13 20:57:23,385 INFO sqlalchemy.engine.Engine SELECT min(addresses.user_id) AS min_1 
FROM addresses
2021-07-13 20:57:23,386 INFO sqlalchemy.engine.Engine [generated in 0.00152s] ()


1

### Alias

In [35]:
a = addresses.alias()
s = select([users]).\
        where(and_(
            users.c.id == a.c.user_id,
            a.c.email_address == 'paco84@hotmail.com'
        ))

conn.execute(s).fetchall()

2021-07-13 20:57:23,418 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.fullname, users.password 
FROM users, addresses AS addresses_1 
WHERE users.id = addresses_1.user_id AND addresses_1.email_address = ?
2021-07-13 20:57:23,420 INFO sqlalchemy.engine.Engine [generated in 0.00174s] ('paco84@hotmail.com',)


[(1, 'Paco', 'Francisco Javier', '25051970')]

### Order_by

In [36]:
s = select([users.c.name]).order_by(users.c.name)

conn.execute(s).fetchall()

2021-07-13 20:57:23,448 INFO sqlalchemy.engine.Engine SELECT users.name 
FROM users ORDER BY users.name
2021-07-13 20:57:23,450 INFO sqlalchemy.engine.Engine [generated in 0.00197s] ()


[('Carla',), ('Lu',), ('Manu',), ('Paco',)]

In [37]:
# En orden descendente
s = select([users.c.name]).order_by(users.c.name.desc())

conn.execute(s).fetchall()

2021-07-13 20:57:23,489 INFO sqlalchemy.engine.Engine SELECT users.name 
FROM users ORDER BY users.name DESC
2021-07-13 20:57:23,491 INFO sqlalchemy.engine.Engine [generated in 0.00216s] ()


[('Paco',), ('Manu',), ('Lu',), ('Carla',)]

### Group_by

In [38]:
s = select([users.c.name, func.count(addresses.c.id)]).\
             select_from(users.join(addresses)).\
             group_by(users.c.name)

conn.execute(s).fetchall()

2021-07-13 20:57:23,520 INFO sqlalchemy.engine.Engine SELECT users.name, count(addresses.id) AS count_1 
FROM users JOIN addresses ON users.id = addresses.user_id GROUP BY users.name
2021-07-13 20:57:23,522 INFO sqlalchemy.engine.Engine [generated in 0.00191s] ()


[('Carla', 1), ('Lu', 1), ('Paco', 1)]

### Having

In [39]:
s = select([users.c.name, func.count(addresses.c.id)]).\
             select_from(users.join(addresses)).\
             group_by(users.c.name).\
             having(func.length(users.c.name) > 4)

conn.execute(s).fetchall()

2021-07-13 20:57:23,555 INFO sqlalchemy.engine.Engine SELECT users.name, count(addresses.id) AS count_1 
FROM users JOIN addresses ON users.id = addresses.user_id GROUP BY users.name 
HAVING length(users.name) > ?
2021-07-13 20:57:23,556 INFO sqlalchemy.engine.Engine [generated in 0.00157s] (4,)


[('Carla', 1)]

### Limit y offset

In [40]:
s = select([users.c.name, addresses.c.email_address]).\
             select_from(users.join(addresses)).\
             limit(2).offset(1)

conn.execute(s).fetchall()

2021-07-13 20:57:23,593 INFO sqlalchemy.engine.Engine SELECT users.name, addresses.email_address 
FROM users JOIN addresses ON users.id = addresses.user_id
 LIMIT ? OFFSET ?
2021-07-13 20:57:23,595 INFO sqlalchemy.engine.Engine [generated in 0.00241s] (2, 1)


[('Lu', 'luci.mendo@gmail.com'), ('Paco', 'paco84@hotmail.com')]

### Union 

In [41]:
from sqlalchemy.sql import union

u = union(
    addresses.select().
             where(addresses.c.email_address.like('%@gmail.com')),
    addresses.select().
             where(addresses.c.email_address.like('%@hotmail.com')),
    ).order_by(addresses.c.email_address)

conn.execute(u).fetchall()

2021-07-13 20:57:23,636 INFO sqlalchemy.engine.Engine SELECT addresses.id, addresses.user_id, addresses.email_address 
FROM addresses 
WHERE addresses.email_address LIKE ? UNION SELECT addresses.id, addresses.user_id, addresses.email_address 
FROM addresses 
WHERE addresses.email_address LIKE ? ORDER BY email_address
2021-07-13 20:57:23,637 INFO sqlalchemy.engine.Engine [generated in 0.00188s] ('%@gmail.com', '%@hotmail.com')


[(1, 2, 'carlota12@gmail.com'),
 (2, 4, 'luci.mendo@gmail.com'),
 (3, 1, 'paco84@hotmail.com')]

### Except

In [42]:
from sqlalchemy.sql import except_

# Devuelve las filas del primer SELECT que no son devuletas por el segundo SELECT 
e = except_(
    addresses.select().
             where(addresses.c.email_address.like('%@gmail.com')),
    addresses.select().
             where(addresses.c.email_address.like('%@hotmail.com')),
    ).order_by(addresses.c.email_address)

conn.execute(e).fetchall()

2021-07-13 20:57:23,673 INFO sqlalchemy.engine.Engine SELECT addresses.id, addresses.user_id, addresses.email_address 
FROM addresses 
WHERE addresses.email_address LIKE ? EXCEPT SELECT addresses.id, addresses.user_id, addresses.email_address 
FROM addresses 
WHERE addresses.email_address LIKE ? ORDER BY email_address
2021-07-13 20:57:23,674 INFO sqlalchemy.engine.Engine [generated in 0.00154s] ('%@gmail.com', '%@hotmail.com')


[(1, 2, 'carlota12@gmail.com'), (2, 4, 'luci.mendo@gmail.com')]

### Join

In [43]:
print(users.join(addresses))

users JOIN addresses ON users.id = addresses.user_id


In [44]:
print(users.join(addresses, addresses.c.email_address.like(users.c.name + '%')))

users JOIN addresses ON addresses.email_address LIKE users.name || :name_1


In [45]:
s = select([users.c.fullname]).select_from(
        users.join(addresses,
        addresses.c.email_address.like(users.c.name + '%'))
        )

conn.execute(s).fetchall()

2021-07-13 20:57:23,758 INFO sqlalchemy.engine.Engine SELECT users.fullname 
FROM users JOIN addresses ON addresses.email_address LIKE users.name || ?
2021-07-13 20:57:23,760 INFO sqlalchemy.engine.Engine [generated in 0.00162s] ('%',)


[('Francisco Javier',), ('Lucia',)]

### Delete

In [46]:
conn.execute(addresses.delete())

2021-07-13 20:57:23,788 INFO sqlalchemy.engine.Engine DELETE FROM addresses
2021-07-13 20:57:23,790 INFO sqlalchemy.engine.Engine [generated in 0.00136s] ()
2021-07-13 20:57:23,791 INFO sqlalchemy.engine.Engine COMMIT


In [47]:
conn.execute(users.delete().where(users.c.name > 'm'))

2021-07-13 20:57:23,823 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.name > ?
2021-07-13 20:57:23,825 INFO sqlalchemy.engine.Engine [generated in 0.00172s] ('m',)
2021-07-13 20:57:23,827 INFO sqlalchemy.engine.Engine COMMIT


## Uso de Clases

### Descripción de tablas con clases 

In [48]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [49]:
from sqlalchemy import Column, Integer, String
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class User(Base):
  __tablename__ = 'users'
  
  id = Column(Integer, primary_key=True)
  name = Column(String)
  fullname = Column(String)
  nickname = Column(String)
  
  def __init__(self, name, fullname, nickname):
    self.name = name
    self.fullname = fullname
    self.nickname = nickname
  
  
class Address(Base):
  __tablename__ = 'addresses'
  
  id = Column(Integer, primary_key=True)
  email_address = Column(String, nullable=False)
  user_id = Column(Integer, ForeignKey('users.id'))
  
  user = relationship("User", backref="addresses")
  
  def __init__(self, email_address, user):
    self.email_address = email_address
    self.user = user

Connexión a la base de datos

In [50]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:',echo=False)

### Create

In [51]:
Base.metadata.create_all(engine) 

###  Creación de sesión



In [52]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

### Insercción de datos

In [53]:
user_1 = User('Carla', 'Carla', '$5&02qq')
user_2 = User('Manu', 'Manuela', 'lqoeoeee')
user_3 = User('Lu', 'Lucia', 'password')

address_1 = Address('carla1?22@gmail.com', user_1)
address_2 = Address('luci.mendo@gmail.com', user_2)
address_3 = Address('paco84@hotmail.com', user_3)

session.add(user_1)
session.add(user_2)
session.add(user_3)

session.add(address_1)
session.add(address_2)
session.add(address_3)

# O
# session.add_all([user_1, user_2, user_3, address_1, address_3, address_3])
session.commit()

### Update

In [54]:
address_3.email_address = 'paco85@hotmail.com'

#Comprobamos si el objeto está en la sesión
print(address_3 in session)
session.commit()

True


#### Cierre de sesión



In [55]:
session.close()

## Querying

In [56]:
for instance in session.query(User).order_by(User.id):
  print(instance.name, instance.fullname)

Carla Carla
Manu Manuela
Lu Lucia


In [57]:
for row in session.query(User, User.name):
  print(row.User, row.name)

<__main__.User object at 0x00000211DD348518> Carla
<__main__.User object at 0x00000211DD348588> Manu
<__main__.User object at 0x00000211DD341DA0> Lu


In [58]:
for user in session.query(User).\
        filter(User.name=='Lu').\
        filter(User.fullname=='Lucia'):
  print(user.name)

Lu


In [59]:
session.query(User).filter(User.name.like('%u')).order_by(User.id)[0].name

'Manu'

In [60]:
#Join
session.query(User).join(Address).filter(Address.email_address=='luci.mendo@gmail.com').all()

### Delete

In [61]:
session.delete(user_1)

In [62]:
session.query(User).filter_by(name='Paco').count()

0

## Ejercicios

1. Defina dos tablas (users y accounts) haciendo uso de SQLAlchemy con los siguientes atributos:
    * users: 
        - user_id: tipo Integer, es primary_key
        - username: tipo String, no pueder ser nulo, tiene que ser único
        - email_address: tipo String, no puede ser nulo
        - phone: tipo String, no puede ser nulo
        - password: tipo String, no puede ser nulo
        - created_on: tipo DateTime, tien por defecto la hora de creación
    * accounts:
        - accounts_id: tipo Integer, es primary_key
        - user_id: es ForeignKey de users.user_id
        - iban: tipo String
        - bic: tipo String
        - money: tipo Numeric con 2 decimales
        
2. Cree las tablas anteriormente definidas con ayuda de un conector SQLite en un fichero de nombre `test.db`. 
3. Introducta al menos 3 valores por cada tabla.
4. Lea todos los valores de las tablas para comprobar se han introducido de forma correcta. 
5. Ordene los usuarios por la cantidad de dinero en cuenta 